In [3]:
import os
import gc
import glob
from datetime import date

import pandas as pd
import numpy as np

from multiprocessing import Pool

from logging import getLogger
logger = getLogger(None)

from tools.datahandling import DataHandling as DH
from tools.util import Util
from tools.encoding import FrequencyEncoder

logfile_name = 'logs/FE_FrequencyEncoding_' + str(date.today().isoformat()) + '.log'
logger = Util.Logger(logfile_name=logfile_name)

In [4]:
train = pd.read_pickle("input/2.initial/train.pkl")
val = pd.read_pickle("input/2.initial/val.pkl")
test = pd.read_pickle("input/2.initial/test.pkl")

In [5]:
columns = train.columns.drop(["click_id", "is_attributed", "data_set", "day", "hour"])

fe = FrequencyEncoder()

for column in columns:
    logger.info('{}'.format(column))
    fe.fit(train[column])
    
    train['FE_' + column] = fe.transform(train[column])
    train = train.drop([column], axis=1)
    
    val['FE_' + column] = fe.transform(val[column])
    val = val.drop([column], axis=1)

    
    test['FE_' + column] = fe.transform(test[column])
    test = test.drop([column], axis=1)

2018-11-11 16:32:06,892 <ipython-input-5-bdd230c74f49> tools.util 6 [INFO][<module>] app 
2018-11-11 16:32:19,535 <ipython-input-5-bdd230c74f49> tools.util 6 [INFO][<module>] channel 
2018-11-11 16:32:39,791 <ipython-input-5-bdd230c74f49> tools.util 6 [INFO][<module>] device 
2018-11-11 16:32:54,581 <ipython-input-5-bdd230c74f49> tools.util 6 [INFO][<module>] ip 
2018-11-11 16:33:18,574 <ipython-input-5-bdd230c74f49> tools.util 6 [INFO][<module>] os 


In [6]:
train = train.drop(['click_id', 'data_set', 'day'], axis=1)
val = val.drop(['click_id', 'data_set', 'day'], axis=1)
test = test.drop(['data_set', 'day'], axis=1)

In [ ]:
train.head()

,is_attributed,hour,FE_app,FE_channel,FE_device,FE_ip,FE_os
68941878,0,0,0.014151,0.030456,0.948708,0.000863,0.215839
68941879,0,0,0.022815,0.040141,0.948708,0.000080,0.016812
68941880,0,0,0.133188,0.028176,0.948708,0.000013,0.215839
68941881,0,0,0.087263,0.030272,0.948708,0.000093,0.240207
68941882,0,0,0.012585,0.005584,0.948708,0.000037,0.048926


In [ ]:
BASE_DIR = 'input/3.base'
ENG_NAME = 'FrequencyEncoding/'

train = DH.change_dtypes(train)
DATA_SET = 'train'
SAVE_DIR = os.path.join(BASE_DIR, ENG_NAME, DATA_SET)
DH.columns2csv(train, SAVE_DIR)
logger.info('data size {}'.format(train.shape))

del train
gc.collect

val = DH.change_dtypes(val)
DATA_SET = 'val'
SAVE_DIR = os.path.join(BASE_DIR, ENG_NAME, DATA_SET)
DH.columns2csv(val, SAVE_DIR)
logger.info('data size {}'.format(val.shape))

del val
gc.collect

test = DH.change_dtypes(test)
DATA_SET = 'test'
SAVE_DIR = os.path.join(BASE_DIR, ENG_NAME, DATA_SET)
DH.columns2csv(test, SAVE_DIR)
logger.info('data size {}'.format(test.shape))

del test
gc.collect

2018-11-11 16:34:02,587 datahandling.py tools.util 28 [INFO][change_dtypes] Converting is_attributed 
2018-11-11 16:34:06,992 datahandling.py tools.util 40 [INFO][change_dtypes] is_attributed has nothing to be done 
2018-11-11 16:34:06,994 datahandling.py tools.util 28 [INFO][change_dtypes] Converting hour 
2018-11-11 16:34:06,995 datahandling.py tools.util 31 [INFO][change_dtypes] hour is converted from int64 to int32 
2018-11-11 16:34:07,298 datahandling.py tools.util 28 [INFO][change_dtypes] Converting FE_app 
2018-11-11 16:34:07,570 datahandling.py tools.util 36 [INFO][change_dtypes] FE_app is converted from float64 to float32 
2018-11-11 16:34:07,571 datahandling.py tools.util 28 [INFO][change_dtypes] Converting FE_channel 
2018-11-11 16:34:07,842 datahandling.py tools.util 36 [INFO][change_dtypes] FE_channel is converted from float64 to float32 
2018-11-11 16:34:07,843 datahandling.py tools.util 28 [INFO][change_dtypes] Converting FE_device 
2018-11-11 16:34:08,121 datahandling.p

In [58]:
my_submission = pd.DataFrame()
my_submission["click_id"] = click["click_id"]
my_submission["is_attributed"] = pred_test
my_submission.to_csv("input/FE_RF_local0888300429641829.csv", index=False)

In [57]:
click.tail()

,click_id,ip,app,device,os,channel,click_time
18790464,18790464,99442,9,1,13,127,2017-11-10 15:00:00
18790465,18790465,88046,23,1,37,153,2017-11-10 15:00:00
18790466,18790467,81398,18,1,17,265,2017-11-10 15:00:00
18790467,18790466,123236,27,1,13,122,2017-11-10 15:00:00
18790468,18790468,73516,12,2,27,265,2017-11-10 15:00:00


In [ ]:
df_train = pd.DataFrame()
df_val   = pd.DataFrame()
df_test  = pd.DataFrame()

for columns in columns_list:
    print(columns)
    he = HashEncoder(n_features=5)
    he.fit(df_selected_train[columns])
    
    df_tmp_train = he.transform(df_selected_train[columns])
    df_train[df_tmp_train.columns] = df_tmp_train
    del df_tmp_train 
    gc.collect()
    
    
    df_tmp_val = he.transform(df_selected_val[columns])
    df_val[df_tmp_val.columns] = df_tmp_val
    del df_tmp_val
    gc.collect()
 
    df_tmp_test = he.transform(df_org_test[columns])
    df_test[df_tmp_test.columns] = df_tmp_test
    del df_tmp_test
    gc.collect()
    
    
df_test["click_id"] = df_org_test["click_id"].astype('int')
df_test["hour"] = df_org_test["hour"].astype('int')
df_train.to_pickle("input/org/HashEncoded/test.pkl")

df_train["hour"] = df_selected_train["hour"].astype('int')
df_train.to_pickle("input/org/HashEncoded/train.pkl")

df_val["hour"] = df_selected_val["hour"]
df_train.to_pickle("input/org/HashEncoded/val.pkl")

In [ ]:
for tr_ind, val_ind in kf.split(all_data):
    X_tr, X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    X_val['item_target_enc'] = X_val['item_id'].map(X_tr.groupby('item_id').target.mean())
    all_data.iloc[val_ind] = X_val

all_data['item_target_enc'].fillna(0.3343, inplace=True) 
encoded_feature = all_data['item_target_enc'].values

In [2]:
df_selected_train = pd.read_pickle("input/org/selected_train.pkl").head(100)
df_selected_val = pd.read_pickle("input/org/selected_val.pkl").head(100)
df_org_test = pd.read_pickle("input/org/org_test.pkl").head(100)

In [ ]:
  
    
    df_val[columns] = me.transform(df_selected_val[columns])
    df_test[columns] = me.transform(df_org_test[columns])

    
df_test["click_id"] = df_org_test["click_id"].astype('int')
df_test["hour"] = df_org_test["hour"].astype('int')
print(df_test)
df_train.to_pickle("input/org/MeanEncoded/test.pkl")

df_train["hour"] = df_selected_train["hour"].astype('int')
print(df_train)
df_train.to_pickle("input/org/MeanEncoded/train.pkl")

df_val["hour"] = df_selected_val["hour"]
df_train.to_pickle("input/org/MeanEncoded/val.pkl")

In [ ]:
df_selected_train = pd.read_pickle("input/org/selected_train.pkl")
df_selected_val = pd.read_pickle("input/org/selected_val.pkl")
df_org_test = pd.read_pickle("input/org/org_test.pkl")

from sklearn.model_selection import KFold
np.random.seed(10)

kf = KFold(n_splits=3, shuffle=False)
columns_list = df_selected_train.columns.drop(["click_id", "is_attributed", "data_set", "day", "hour"])

for columns in columns_list:
    print(columns)
    dfs = []
    for tr_ind, val_ind in kf.split(df_selected_train[columns]):
        X_tr, X_val = df_selected_train[columns].iloc[tr_ind], df_selected_train[columns].iloc[val_ind]
        y_tr, y_val= df_selected_train['is_attributed'].iloc[tr_ind], df_selected_train['is_attributed'].iloc[val_ind]
        
        me = MeanEncoder()
        me.fit(X_tr, y_tr)
        me.transform(X_val)
        dfs.append(me.transform(X_val))
    
    df_selected_train["ME_" + columns] = pd.concat(dfs, axis=0)
    df_selected_val["ME_" + columns] = me.transform(df_selected_val[columns])
    df_org_test["ME_" + columns] = me.transform(df_org_test[columns])
    
df_train = df_selected_train.drop(columns_list, axis=1)
df_train = df_train.drop(["click_id", "data_set", "is_attributed"], axis=1)


df_val = df_selected_val.drop(columns_list, axis=1)
df_val = df_val.drop(["click_id", "data_set", "is_attributed"], axis=1)


df_test = df_org_test.drop(columns_list, axis=1)
df_test = df_test.drop(["click_id", "data_set", "is_attributed"], axis=1)


df_test["click_id"] = df_org_test["click_id"].astype('int')
df_test["hour"] = df_org_test["hour"].astype('int')
df_test.to_pickle("input/org/MeanEncoded/test.pkl")

df_train["hour"] = df_selected_train["hour"].astype('int')
df_train.to_pickle("input/org/MeanEncoded/train.pkl")

df_val["hour"] = df_selected_val["hour"]
df_tal.to_pickle("input/org/MeanEncoded/val.pkl")

app
channel
device
ip


In [50]:
df_test.head()

,day,hour,ME_app,ME_channel,ME_device,ME_ip,ME_os,click_id
0,10,4,0.0,0.0,0,0.0,0.0,0
1,10,4,0.0,0.0,0,0.0,0.0,1
2,10,4,0.0,0.0,0,0.0,0.0,2
3,10,4,0.0,0.0,0,0.0,0.0,3
4,10,4,0.0,0.0,0,0.0,0.0,4


In [48]:
df_org_test

,app,channel,click_id,data_set,device,ip,is_attributed,os,day,hour,ME_app,ME_channel,ME_device,ME_ip,ME_os
0,9,107,0.0,1,1,5744,999,3,10,4,0.0,0.0,0,0.0,0.0
1,9,466,1.0,1,1,119901,999,3,10,4,0.0,0.0,0,0.0,0.0
2,21,128,2.0,1,1,72287,999,19,10,4,0.0,0.0,0,0.0,0.0
3,15,111,3.0,1,1,78477,999,13,10,4,0.0,0.0,0,0.0,0.0
4,12,328,4.0,1,1,123080,999,13,10,4,0.0,0.0,0,0.0,0.0
5,18,107,5.0,1,1,110769,999,13,10,4,0.0,0.0,0,0.0,0.0
6,3,137,6.0,1,1,12540,999,1,10,4,0.0,0.0,0,0.0,0.0
7,27,153,7.0,1,1,88637,999,19,10,4,0.0,0.0,0,0.0,0.0
8,18,107,9.0,1,1,14932,999,10,10,4,0.0,0.0,0,0.0,0.0
9,12,424,8.0,1,1,123701,999,53,10,4,0.0,0.0,0,0.0,0.0
